In [8]:
import numpy as np
import os
import pandas as pd
from scipy.interpolate import RBFInterpolator
import matplotlib.pyplot as plt

In [9]:
# Folder path containing CSV files
folder_path = "data/csv"
out_file    = "results/losses/losses.csv"

# Ensure the output directory exists
os.makedirs(os.path.dirname(out_file), exist_ok=True)
write_header = not os.path.exists(out_file)

# Define value ranges
first_values = np.arange(0.3, 0.7, 0.05)  # 0.25 to 0.7 with step 0.05
# second_values = np.arange(3, 15, 1)  # 3 to 14
second_values = np.arange(3, 15, 1)  # 3 to 14

In [10]:
def numerical_gradient(f_interp, pts, h=1e-4):
    grads = []
    for i in range(2):
        offset = np.zeros_like(pts)
        offset[:, i] = h
        grad_i = (f_interp(pts + offset) - f_interp(pts - offset)) / (2 * h)
        grads.append(grad_i)
    return grads

def numerical_laplacian(f_interp, pts, h=1e-4):
    lap = np.zeros(len(pts))
    for i in range(2):
        offset = np.zeros_like(pts)
        f_plus = f_interp(pts + offset)
        f_minus = f_interp(pts - offset)
        f_center = f_interp(pts)
        lap += (f_plus - 2 * f_center + f_minus) / h**2
    return lap

In [ ]:
for first in first_values:
    for second in second_values:
        first_str = f"{first:.2f}".rstrip("0").rstrip(".")
        file_name = f"W_{first_str}_{second}_1.csv"
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            
            # Load CSV file
            df = pd.read_csv(file_path)
            
            # Extract input and output data
            points = df[['x', 'y']].values
            u = df['u'].values
            v = df['v'].values
            p = df['p'].values

            # Interpolators
            rbf_u = RBFInterpolator(points, u, kernel='thin_plate_spline')
            rbf_v = RBFInterpolator(points, v, kernel='thin_plate_spline')
            rbf_p = RBFInterpolator(points, p, kernel='thin_plate_spline')

            # Derivatives
            du_dx, du_dy = numerical_gradient(rbf_u, points)
            dv_dx, dv_dy = numerical_gradient(rbf_v, points)
            dp_dx, dp_dy = numerical_gradient(rbf_p, points)
            lap_u = numerical_laplacian(rbf_u, points)
            lap_v = numerical_laplacian(rbf_v, points)

            # Residuals
            mu = 0.01
            cont_res = du_dx + dv_dy
            mom_x_res = u * du_dx + v * du_dy + dp_dx - mu * lap_u
            mom_y_res = u * dv_dx + v * dv_dy + dp_dy - mu * lap_v

            # Plotting
            fig, axs = plt.subplots(1, 3, figsize=(18, 5))

            sc1 = axs[0].scatter(points[:, 0], points[:, 1], c=cont_res, cmap='coolwarm')
            axs[0].set_title('Continuity Residual')
            plt.colorbar(sc1, ax=axs[0])

            sc2 = axs[1].scatter(points[:, 0], points[:, 1], c=mom_x_res, cmap='coolwarm')
            axs[1].set_title('X-Momentum Residual')
            plt.colorbar(sc2, ax=axs[1])

            sc3 = axs[2].scatter(points[:, 0], points[:, 1], c=mom_y_res, cmap='coolwarm')
            axs[2].set_title('Y-Momentum Residual')
            plt.colorbar(sc3, ax=axs[2])

            for ax in axs:
                ax.set_xlabel('x')
                ax.set_ylabel('y')
                ax.set_aspect('equal')

            plt.tight_layout()
            plt.savefig(f"results/losses/residuals_{first_str}_{second}.png")

            # Mean absolute residuals
            avg_continuity = np.mean(np.abs(cont_res))
            avg_momentum_x = np.mean(np.abs(mom_x_res))
            avg_momentum_y = np.mean(np.abs(mom_y_res))

            row = {
                'f': first,
                'n': second,
                'continuity': avg_continuity,
                'momentum_x': avg_momentum_x,
                'momentum_y': avg_momentum_y
            }
            
            pd.DataFrame([row]).to_csv(
                out_file,
                mode='a',
                header=write_header,
                index=False
            )
            write_header = False  # after first write, never write header again

Processing W_0.3_3_1.csv...


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
import os
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def fit_tps_rbf(pts, vals, smoothing=0.0):
    """
    Thin‑plate‑spline RBF solve on GPU with float32 everywhere.
    """
    N = pts.shape[0]
    # pairwise distances
    D = torch.cdist(pts, pts) + 1e-12
    K = D**2 * torch.log(D)                             # (N,N)
    P = torch.cat([torch.ones(N,1, device=device), pts], dim=1)  # (N,3)

    # build system matrix
    L = N + P.shape[1]
    A = torch.zeros(L, L, device=device, dtype=pts.dtype)
    A[:N, :N]   = K + smoothing * torch.eye(N, device=device, dtype=pts.dtype)
    A[:N, N:]   = P
    A[N:, :N]   = P.T

    # rhs: vals must match dtype
    rhs = torch.cat([
        vals,
        torch.zeros(P.shape[1], device=device, dtype=pts.dtype)
    ])

    sol = torch.linalg.solve(A, rhs)
    return sol[:N], sol[N:]

def eval_tps_rbf(pts_src, w, c, pts_q):
    Dq = torch.cdist(pts_q, pts_src) + 1e-12   # (M,N)
    Φ  = Dq**2 * torch.log(Dq)                 # (M,N)
    Pq = torch.cat([torch.ones(pts_q.shape[0],1,device=device, dtype=pts_q.dtype),
                    pts_q], dim=1)              # (M,3)
    return Φ @ w + Pq @ c

def compute_derivatives(pts_src, w, c, pts, h=1e-4):
    f0  = eval_tps_rbf(pts_src, w, c, pts)
    ex  = torch.tensor([h,0.], device=device, dtype=pts.dtype)
    ey  = torch.tensor([0.,h], device=device, dtype=pts.dtype)
    fxp = eval_tps_rbf(pts_src, w, c, pts + ex)
    fxm = eval_tps_rbf(pts_src, w, c, pts - ex)
    fyp = eval_tps_rbf(pts_src, w, c, pts + ey)
    fym = eval_tps_rbf(pts_src, w, c, pts - ey)

    d_x = (fxp - fxm) / (2*h)
    d_y = (fyp - fym) / (2*h)
    lap = (fxp + fxm + fyp + fym - 4*f0) / (h*h)
    return d_x, d_y, lap

def process_all(folder_in, folder_out,
                first_vals=np.arange(0.3,0.7,0.05),
                second_vals=np.arange(3,15),
                mu=0.01, h=1e-4):

    img_dir = os.path.join(folder_out, "residuals")
    os.makedirs(img_dir, exist_ok=True)

    records = {'f':[], 'n':[], 'continuity':[], 'momentum_x':[], 'momentum_y':[]}

    for f in first_vals:
        fstr = f"{f:.2f}".rstrip("0").rstrip(".")
        for n in second_vals:
            fname = f"W_{fstr}_{n}_1.csv"
            path  = os.path.join(folder_in, fname)
            if not os.path.isfile(path): continue
            print("GPU processing", fname)

            # load data and cast to float32 on GPU
            df     = pd.read_csv(path, usecols=['x','y','u','v','p'])
            pts_np = df[['x','y']].to_numpy(dtype=np.float32)
            u_np   = df['u'].to_numpy(dtype=np.float32)
            v_np   = df['v'].to_numpy(dtype=np.float32)
            p_np   = df['p'].to_numpy(dtype=np.float32)

            pts = torch.from_numpy(pts_np).to(device=device, dtype=torch.float32)
            u   = torch.from_numpy(u_np).to( device=device, dtype=torch.float32)
            v   = torch.from_numpy(v_np).to( device=device, dtype=torch.float32)
            p   = torch.from_numpy(p_np).to( device=device, dtype=torch.float32)

            # fit and derive
            wu, cu = fit_tps_rbf(pts, u)
            wv, cv = fit_tps_rbf(pts, v)
            wp, cp = fit_tps_rbf(pts, p)

            du_dx, du_dy, lap_u = compute_derivatives(pts, wu, cu, pts, h)
            dv_dx, dv_dy, lap_v = compute_derivatives(pts, wv, cv, pts, h)
            dp_dx, dp_dy, _     = compute_derivatives(pts, wp, cp, pts, h)

            cont = du_dx + dv_dy
            momx = u*du_dx + v*du_dy + dp_dx - mu*lap_u
            momy = u*dv_dx + v*dv_dy + dp_dy - mu*lap_v

            # back to CPU for plotting
            xy   = pts.cpu().numpy()
            C    = cont.cpu().numpy()
            MX   = momx.cpu().numpy()
            MY   = momy.cpu().numpy()

            fig, ax = plt.subplots(1,3,figsize=(18,5))
            for res, title, a in zip([C, MX, MY],
                                     ["Continuity","X‑Momentum","Y‑Momentum"],
                                     ax):
                im = a.scatter(xy[:,0], xy[:,1], c=res, cmap='coolwarm', s=5)
                a.set(aspect='equal', title=f"{title} Residual",
                      xlabel='x', ylabel='y')
                fig.colorbar(im, ax=a)
            plt.tight_layout()
            fig.savefig(os.path.join(img_dir, f"gpu_res_{fstr}_{n}.png"))
            plt.close(fig)

            records['f'].append(f)
            records['n'].append(n)
            records['continuity'].append(np.mean(np.abs(C)))
            records['momentum_x'].append(np.mean(np.abs(MX)))
            records['momentum_y'].append(np.mean(np.abs(MY)))

            del pts,u,v,p,wu,cu,wv,cv,wp,cp
            torch.cuda.empty_cache()

    pd.DataFrame(records).to_csv(os.path.join(folder_out, "losses_gpu.csv"), index=False)


if __name__=="__main__":
    process_all("data/csv", "results/losses_gpu")


GPU processing W_0.3_3_1.csv
GPU processing W_0.3_4_1.csv
GPU processing W_0.3_5_1.csv
GPU processing W_0.3_6_1.csv
